# 4D-STEM Notebook
In this Notebook it is demonstrated how to work with big 4D-STEM data, especially when applying Canny edge detection. The goal is to eliminate features observed within the transmitted beam disk and separate this from the actual displacement of the disk. If the analysis is correct, the result should be a map showing the disk displacement in x- and y-direction.

## 0 Import repositories

In [1]:
%matplotlib qt

In [6]:
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import pyxem as pxm
from pyxem.utils.io_utils import _parse_hdr
from skimage import data, io, feature
from converter_nord import save_signal
import zarr
import dask.array as da
import os
from scipy import ndimage, misc

In [ ]:
!pip list

## <br><br>

## 1 Loading

In [10]:
data_path = "../../Duncan_Alexander/Merlin detector/22-05-06 PTO_DSO_CL02/Scan4D_02.mib"

In [11]:
_parse_hdr(data_path)

{'width': 256,
 'height': 256,
 'Assembly Size': '1x1',
 'offset': 384,
 'data-type': 'unsigned',
 'data-length': '16',
 'Counter Depth (number)': 12,
 'raw': 'MIB',
 'byte-order': 'dont-care',
 'record-by': 'image',
 'title': '',
 'date': '20220506',
 'time': '14:35:11.838851',
 'data offset': 384}

In [14]:
os.getcwd()

'Z:\\Federico_Garro\\Pixelated detector notebooks'

In [15]:
save_signal(os.getcwd(),'4D_converted_4.hspy',384,128,1)

Z:\Federico_Garro\Pixelated detector notebooks already exist


In [5]:
s = hs.load('4D_converted.hspy', lazy=True)

In [9]:
s.data

,Array,Chunk
Bytes,6.00 GiB,2.00 MiB
Shape,"(128, 384, 256, 256)","(32, 32, 32, 32)"
Count,3073 Tasks,3072 Chunks
Type,uint16,numpy.ndarray


In [ ]:
s.save('4D_converted.zspy',chunks=(32,32,32,32))

In [161]:
s = hs.load("4D_converted.zspy", lazy=True)

In [144]:
s.data

,Array,Chunk
Bytes,6.00 GiB,2.00 MiB
Shape,"(128, 384, 256, 256)","(32, 32, 32, 32)"
Count,3073 Tasks,3072 Chunks
Type,uint16,numpy.ndarray


In [45]:
s.axes_manager.gui()

In [145]:
s.plot()

[########################################] | 100% Completed |  1.4s
[########################################] | 100% Completed |  1.5s
[########################################] | 100% Completed |  1.5s


In [8]:
s.inav[0,0].save('Disk.tif')

Overwrite 'C:\Users\hedda\OneDrive - NTNU\Dokumenter\EPFL\V22 [Semester Project] 4D-STEM\Measurements\060522_4D-STEM\02\Disk.tif' (y/n)?
 y


In [116]:
s_ax = s_corr

for i in range(4):
    if i < 2:
        s_ax.axes_manager[i].units = "nm"
        s_ax.axes_manager[i].scale = 0.49907
    else:
        s_ax.axes_manager[i].units = "mrad"
        s_ax.axes_manager[i].scale = 0.032484

s_ax.axes_manager[0].name = "Length"
s_ax.axes_manager[1].name = "Height"

s_ax.axes_manager[2].name = "x"
s_ax.axes_manager[3].name = "y"

s_ax.axes_manager

s_corr = s_ax

In [9]:
s_rebin = s.rebin(scale=(2, 2, 2, 2), rechunk=False)

In [10]:
s_rebin.compute()

[########################################] | 100% Completed | 18.2s


In [16]:
s_rebin

<ElectronDiffraction2D, title: , dimensions: (192, 64|256, 256)>

In [9]:
s_rebin.plot()

In [96]:
s_rebin_T = s_rebin.T
s_rebin_T.plot()

## Correcting dead and hot pixels

In [1]:
s_dead_pixels = s_rebin.find_dead_pixels(dead_pixel_value=0)

NameError: name 's_rebin' is not defined

In [122]:
s_hot_pixels = s_rebin.find_hot_pixels(show_progressbar=True, threshold_multiplier = 10)

In [123]:
s_corr = s_rebin.correct_bad_pixels(s_dead_pixels+s_hot_pixels, show_progressbar=True, inplace=False, lazy_result=True)

In [118]:
s_corr.T.inav[105,134].plot()

In [21]:
s_corr.inav


KeyboardInterrupt



## Virtual filters

In [74]:
hs.print_known_signal_types()

signal_type,aliases,class name,package
correlation,,Correlation2D,pyxem
dpc,,DPCBaseSignal,pyxem
dpc,,DPCSignal1D,pyxem
dpc,,DPCSignal2D,pyxem
DielectricFunction,dielectric function,DielectricFunction,hyperspy
diffraction,,Diffraction1D,pyxem
diffraction,,Diffraction2D,pyxem
diffraction_variance,,DiffractionVariance1D,pyxem
diffraction_variance,,DiffractionVariance2D,pyxem
diffraction_vectors,,DiffractionVectors,pyxem


In [132]:
s_corr.plot()

In [66]:
adf_roi = hs.roi.CircleROI(cx=64, cy=64, r=35, r_inner=20)

In [84]:
s_corr.compute()

[########################################] | 100% Completed |  1min 20.6s


In [85]:
s_bf = s_corr.virtual_bright_field(128, 128, 85, show_progressbar=True)
s_corr.plot(navigator=s_bf) 

AttributeError: 'Diffraction2D' object has no attribute 'virtual_bright_field'

In [67]:
s_adf = adf_roi(s_rebin, axes=(2,3))
s_adf_sum = s_adf.nansum(axis=(2,3), rechunk=False)
s_adf_sum = s_adf_sum.T

In [69]:
s

<LazyElectronDiffraction2D, title: , dimensions: (384, 128|256, 256)>

In [68]:
s_adf_sum.compute()

AttributeError: 'ElectronDiffraction2D' object has no attribute 'compute'

In [ ]:
s_rebin.navigator = s_adf_sum

In [ ]:
s_rebin.plot(norm="symlog")

In [38]:
s_corr

<LazyDiffraction2D, title: , dimensions: (192, 64|128, 128)>

In [63]:
s.compute()

[##############################          ] | 77% Completed |  2min 21.7s


OSError: Can't read data (memory allocation failed for shuffle buffer)

In [62]:
s_bf = s.virtual_bright_field(128, 128, 85, show_progressbar=True)
s.plot(navigator=s_bf)

AttributeError: 'LazyElectronDiffraction2D' object has no attribute 'virtual_bright_field'

In [110]:
s_bf = s_corr.lazy_virtual_bright_field(128, 128, 85, show_progressbar=True)
s_corr.plot(navigator=s_bf)

C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\pyxem\signals\diffraction2d.py:1502: FutureWarning: This method is depreacted and will be removed in the next version
  warnings.warn("This method is depreacted and will be removed in the next version",FutureWarning)


[######                                  ] | 17% Completed |  0.1s

C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\hyperspy\_signals\lazy.py:251: VisibleDeprecationWarning: The `progressbar` keyword is deprecated and will be removed in HyperSpy 2.0. Use `show_progressbar` instead.
  warnings.warn(


[########################################] | 100% Completed |  1.6s


In [113]:
s_corr.plot(navigator=s_bf)

AttributeError: 'NoneType' object has no attribute 'T'

## Sobel filter

In [162]:
s_sobel = s.map(ndimage.sobel,mode='nearest')

[########################################] | 100% Completed |  1.8s


## Apply canny filter

In [124]:
s_out = s_corr.map(feature.canny, high_threshold = 250, low_threshold=10, sigma = 10, show_progressbar = True, inplace = False, lazy_output=True)

[########################################] | 100% Completed |  1.6s


## Center of mass

In [126]:
s_out.plot()

[########################################] | 100% Completed |  5min 10.1s
[########################################] | 100% Completed |  5min 10.1s
[########################################] | 100% Completed |  3.6s
[########################################] | 100% Completed |  3.5s
[########################################] | 100% Completed |  3.3s


In [138]:
s_out.plot()

[########################################] | 100% Completed |  3.5s


In [125]:
s_com = s_out.center_of_mass()
s_com.plot()

[#######################                 ] | 57% Completed |  2min 42.2s

C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\dask\array\core.py:4743: RuntimeWarning: invalid value encountered in true_divide
  result = function(*args, **kwargs)


[########################################] | 100% Completed |  4min 40.1s
[########################################] | 100% Completed |  4min 40.1s


In [223]:
s_com.T.plot(cmap='viridis')

In [97]:
s_com.navigator

In [102]:
s.isig[:].sum(-1).plot()

[########################################] | 100% Completed |  2.5s


In [104]:
s_abf = s.lazy_virtual_annular_dark_field(128, 128, 40, 80, show_progressbar=True)
s.plot(navigator=s_abf)

C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\pyxem\signals\diffraction2d.py:1546: FutureWarning: This method is depreacted and will be removed in the next version
  warnings.warn("This method is depreacted and will be removed in the next version",FutureWarning)
C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\hyperspy\_signals\lazy.py:251: VisibleDeprecationWarning: The `progressbar` keyword is deprecated and will be removed in HyperSpy 2.0. Use `show_progressbar` instead.
  warnings.warn(


[########################################] | 100% Completed | 13.8s


In [111]:
s.plot(navigator=s_abf)

AttributeError: 'NoneType' object has no attribute 'sum'

In [123]:
s

<LazyElectronDiffraction2D, title: , dimensions: (384, 128|256, 256)>

In [107]:
s_rebin.plot()

In [126]:
s.navigator.plot()

In [147]:
adf_roi = hs.roi.CircleROI(cx=128, cy=128, r=50, r_inner=0)

In [151]:
adf_roi

AttributeError: 'CircleROI' object has no attribute 'plot'

In [179]:
s_adf = adf_roi(s, axes=(2, 3))
s_adf_sum = s_adf.nansum(axis=(2, 3), rechunk=False)
s_adf_sum = s_adf_sum.T

In [180]:
s_adf_sum.compute()

[########################################] | 100% Completed | 10.5s


In [187]:
s.signal = s_adf_sum

In [188]:
s.plot(norm="symlog")

In [189]:
s_adf_sum.plot()

In [192]:
image_ROI = adf_roi.interactive(s_adf_sum)

In [153]:
# Create an interactive ROI, show it on the previous adf image
# Calculate the PACBED pattern from the ROI, plot 

s_bf_ROI=hs.roi.RectangularROI(left=0.5, right=1.0, top=0.5, bottom=1.0)
image_ROI=s_bf_ROI.interactive(s_bf)

In [194]:
adf_roi(s_adf_sum).plot()

### ROI

In [195]:
s.sum().compute()

[########################################] | 100% Completed | 11.1s


In [41]:
s.plot()

In [42]:
roi = hs.roi.CircleROI(cx=8.3/2, cy=8.3/2, r=2.0, r_inner=0)

In [12]:
s_roi = roi(s_corr)

In [13]:
s_roi.plot()

[########################################] | 100% Completed |  1.6s


In [92]:
s.T.plot()

[########################################] | 100% Completed |  0.6s


In [100]:
s_corr.inav[77,46].plot()

In [91]:

fig,ax = plt.subplots(figsize=(10,10))

for dx in range(20):
    x = 150 + dx
    y = 16
    ax.imshow(s_corr.inav[x,y],cmap='viridis')
    plt.tight_layout()
    plt.axis('off')
    plt.savefig('a_cross_'+str(x)+'_'+str(y)+'.png')
    




In [44]:
s_adf = roi(s, axes=(2, 3))
s_adf_sum = s_adf.nansum(axis=(2, 3), rechunk=False)
s_adf_sum = s_adf_sum.T

C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\dask\array\core.py:4656: PerformanceWarning: Increasing number of chunks by factor of 48
  result = blockwise(


In [45]:
s_adf_sum.compute()

KeyboardInterrupt: 

In [39]:
s.navigator = s_adf_sum

In [ ]:
s_adf_sum.plot()

AttributeError: 'NoneType' object has no attribute 'data'

In [178]:
roi = hs.roi.CircleROI(cx = 32, cy=32, r = 20, r_inner=0)
s.plot(navigator_dimension=1) # plot signal to have where to display the widget
imr = roi.interactive(s.T, navigation_signal=s, color="red")
roi(imr).plot()

AttributeError: 'AxesImage' object has no property 'navigator_dimension'

In [139]:
nav.plot()

AttributeError: 'NoneType' object has no attribute 'plot'

In [143]:
imr = roi.interactive(s,navigation_signal=nav,color='red')

In [144]:
imr.plot()

[########################################] | 100% Completed |  0.1s


In [204]:
s.plot_integrated_intensity(roi)

KeyboardInterrupt: 

In [205]:
vdf = s.get_integrated_intensity(roi)

I want to:
* COM with scikit edge detection
* remove dead pixels (remove hot pixels)
* BF image (virtual detector) from 4D-STEM from month 04 
* Strain stuff, read article from Wei
* Annotate figures like Wei, combine 4D with HR

## General commands

#### Crop 4D data

In [28]:
s_new = s.inav[30:70,30:70]

#### Image of diffraction disk (signal dimension)

In [51]:
s.inav[10,10].plot()
# s.inav[10,10].save('Disk.png')

#### Image of sample (navigator dimension)

In [41]:
s.isig[10,10].plot()
# s.isig[10,10].save('Sample.png')

#### Access axes manager

In [14]:
s.axes_manager.gui()

#### Access data information 

In [33]:
s.data

,Array,Chunk
Bytes,2.00 GiB,16.00 MiB
Shape,"(128, 256, 256, 256)","(16, 16, 256, 256)"
Count,14084364 Tasks,128 Chunks
Type,bool,numpy.ndarray


#### Convert to lazy signal
OBS: changes the signal class 

In [66]:
s = hs.signals.Signal2D(s_out).as_lazy()

#### Check chunksize

In [69]:
s.data.chunksize

(32, 32, 32, 32)

In [ ]:
s_com = s.center_of_mass(threshold=20, mask=None, lazy_result=True, show_progressbar=True, chunk_calculations=(16,16,16,16))

In [ ]:
s_com

In [ ]:
s_com.save('com.tif')

####  ADF

In [ ]:
s_adf = s_.lazy_virtual_annular_dark_field(25, 25, 5, 20, show_progressbar=True)
s.plot(navigator=s_adf)

C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\pyxem\signals\diffraction2d.py:1546: FutureWarning: This method is depreacted and will be removed in the next version
  warnings.warn("This method is depreacted and will be removed in the next version",FutureWarning)
C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\hyperspy\_signals\lazy.py:111: VisibleDeprecationWarning: The `progressbar` keyword is deprecated and will be removed in HyperSpy 2.0. Use `show_progressbar` instead.
  warnings.warn(


[###########################             ] | 68% Completed |  1min 48.5s

#### BF

In [ ]:
s_bf = s.lazy_virtual_bright_field(25, 25, 5, show_progressbar=True)
s.plot(navigator=s_bf)

#### Center of mass

In [ ]:
s_com = s_out.center_of_mass(threshold=2, show_progressbar=True)
s_com.plot()

[##                                      ] | 6% Completed |  4min 11.1s

#### Canny filter

In [62]:
edge = feature.canny(s_corr.inav[50,50].data, sigma = 4, high_threshold = 70, low_threshold = 10)

#### Plotting figures in grid

In [67]:
fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(6,6))



edges1 = s_out.inav[70,20]
edges2 = s_out.inav[85,20]
edges3 = s_out.inav[130,20]
edges4 = s_out.inav[165,20]


ax[0,0].imshow(edges1,cmap='viridis')
ax[0,1].imshow(edges2,cmap='viridis')
ax[1,0].imshow(edges3,cmap='viridis')
ax[1,1].imshow(edges4,cmap='viridis')


for i in range(2):
    for j in range(2):
        ax[i,j].axis('off')

fig.tight_layout()
plt.show()    

In [65]:
fig,ax = plt.subplots(ncols=3,nrows=3,figsize=(6,6))

edges1 = feature.canny(s_corr.inav[50,50].data,sigma=1, low_threshold=10, high_threshold=250)
edges2 = feature.canny(s_corr.inav[50,50].data,sigma=5, low_threshold=10, high_threshold=250)
edges3 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=250)


edges4 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=0, high_threshold=250)
edges5 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=250)
edges6 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=200, high_threshold=250)


edges7 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=50)
edges8 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=250)
edges9 = feature.canny(s_corr.inav[50,50].data,sigma=10, low_threshold=10, high_threshold=500)



# edges1 = s_out.inav[0,1]
# edges2 = s_out.inav[15,19]
# edges3 = s_out.inav[2,20]
# edges4 = s_out.inav[20,40]

# ax[0].imshow(edges1,cmap='gray')
# ax[1].imshow(edges2,cmap='gray')
# ax[2].imshow(edges3,cmap='gray')
# ax[3].imshow(edges4,cmap='gray')

ax[0,0].imshow(edges1,cmap='gray')
ax[0,1].imshow(edges2,cmap='gray')
ax[0,2].imshow(edges3,cmap='gray')

ax[1,0].imshow(edges4,cmap='gray')
ax[1,1].imshow(edges5,cmap='gray')
ax[1,2].imshow(edges6,cmap='gray')

ax[2,0].imshow(edges7,cmap='gray')
ax[2,1].imshow(edges8,cmap='gray')
ax[2,2].imshow(edges9,cmap='gray')

for i in range(3):
    for j in range(3):
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])

ax[0,0].set_ylabel('$I_{low}$=10, $I_{high}$=250')    
ax[1,0].set_ylabel('$\sigma$=10, $I_{high}$=250')    
ax[2,0].set_ylabel('$\sigma$=10, $I_{low}$=10')    

# plt.axis('off')
fig.tight_layout()
plt.show()    
    

In [66]:
print('OK')

OK


In [25]:
fig,ax = plt.subplots(ncols=2,nrows=2,figsize=(6,6))
ax[0,0].imshow(s_out.inav[70,20])
ax[0,1].imshow(s_out.inav[85,20])
ax[1,0].imshow(s_out.inav[130,20])
ax[1,1].imshow(s_out.inav[165,20])

C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\ipykernel\eventloops.py:105: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  app.exec_()
C:\Users\hedda\Anaconda3\envs\hspy_environment\lib\site-packages\ipykernel\eventloops.py:105: UserWarning: Attempted to set non-positive top ylim on a log-scaled axis.
Invalid limit will be ignored.
  app.exec_()
